# Re-Analyzing the Compas Dataset (Focusing on Logistic Regression)

In this notebook, I replicate the logistic regression component of ProPublica's analysis, which isolates the predictive value of various demographic indicators. As discussed below, ProPublica's statistical analysis is mathematically accurate but somewhat misleading: instead of comparing demographic indicators using the *entire sample*, the ProPublica analysis limits its comparisions to very specific and small subsets of the overall sample.

When their analysis is properly extended to the entire sample, several striking claims become significantly more modest:

* Black defendants were **~~45 percent~~ 20 percent** more likely to be assigned higher risk scores than white defendants.

* Women are **~~19.4%~~ 8%** more likely than men to get a higher score.

* [P]eople under 25 are **~~2.5~~ 1.9** times as likely to get a higher score as middle aged defendants.

* The violent score overpredicts recidivism for black defendants by **~~77.3%~~ 33%** compared to white defendants.

* Defendands under 25 are **~~7.4~~ 3.8** times as likely to get a higher score as middle aged defendants.

Whenever possible, I replicate analyses using the [code provided by ProPublica](https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb).

Finally, a brief note on terminology before we proceed. The ProPublica authors repeatedly uses "percent change" (%) as a metric. An alternative is "percentage point" (pp). Wherever possible, I try to display both. Here is an example of how they are related: 

* Imagine that Person A's likelihood of being classified as high-risk is 25%, and Person B's likelihood is 50%. This represents a 100% change (`(50 - 25)/25`) and a 25pp (`50 - 25`) change. 

* Imagine that Person A's likelihood of being classified as high-risk is 1%, and Person B's likelihood is 2%. This is also a 100% change but only a 1pp change.

Without understanding the "base rate", i.e., Person A's likelihood, reporting only percent change can be somewhat misleading.

# Claim 1: Black defendants were 45 percent more likely to be assigned higher risk scores than white defendants

When recalculated over the entire same rather than a specific subset, this value is approximately 20%.

First, we'll replicate what the author did.

In [2]:
# The code below is copied from the original

# Cell 2

library(dplyr)
library(ggplot2)
raw_data <- read.csv("./compas-scores-two-years.csv")
nrow(raw_data)

# Cell 3

df <- dplyr::select(raw_data, age, c_charge_degree, race, age_cat, score_text, sex, priors_count, 
                    days_b_screening_arrest, decile_score, is_recid, two_year_recid, c_jail_in, c_jail_out) %>% 
        filter(days_b_screening_arrest <= 30) %>%
        filter(days_b_screening_arrest >= -30) %>%
        filter(is_recid != -1) %>%
        filter(c_charge_degree != "O") %>%
        filter(score_text != 'N/A')
nrow(df)

# Cell 4

df$length_of_stay <- as.numeric(as.Date(df$c_jail_out) - as.Date(df$c_jail_in))

# Cell 5 through 15 ommitted - they display summary statistics only
# No changes are made to the dataset

# Cell 16

df <- mutate(df, crime_factor = factor(c_charge_degree)) %>%
      mutate(age_factor = as.factor(age_cat)) %>%
      within(age_factor <- relevel(age_factor, ref = 1)) %>%
      mutate(race_factor = factor(race)) %>%
      within(race_factor <- relevel(race_factor, ref = 3)) %>%
      mutate(gender_factor = factor(sex, labels= c("Female","Male"))) %>%
      within(gender_factor <- relevel(gender_factor, ref = 2)) %>%
      mutate(score_factor = factor(score_text != "Low", labels = c("LowScore","HighScore")))
model <- glm(score_factor ~ gender_factor + age_factor + race_factor +
                            priors_count + crime_factor + two_year_recid, family="binomial", data=df)
summary(model)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



[1] 7214

[1] 6172


Call:
glm(formula = score_factor ~ gender_factor + age_factor + race_factor + 
    priors_count + crime_factor + two_year_recid, family = "binomial", 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.9966  -0.7919  -0.3303   0.8121   2.6024  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -1.52554    0.07851 -19.430  < 2e-16 ***
gender_factorFemale          0.22127    0.07951   2.783 0.005388 ** 
age_factorGreater than 45   -1.35563    0.09908 -13.682  < 2e-16 ***
age_factorLess than 25       1.30839    0.07593  17.232  < 2e-16 ***
race_factorAfrican-American  0.47721    0.06935   6.881 5.93e-12 ***
race_factorAsian            -0.25441    0.47821  -0.532 0.594717    
race_factorHispanic         -0.42839    0.12813  -3.344 0.000827 ***
race_factorNative American   1.39421    0.76612   1.820 0.068784 .  
race_factorOther            -0.82635    0.16208  -5.098 3.43e-07 ***
priors_count  

Note that the model above perfectly reproduces the model in the [original](http://localhost:8888/notebooks/Compas%20Analysis.ipynb). See output of Cell 16. The author then writes

> Black defendants are 45% more likely than white defendants to receive a higher score correcting for the seriousness of their crime, previous arrests, and future criminal behavior.

In cell 17, the author runs the following calculation to arrive at this 45% figure.

In [3]:
# Cell 17
control <- exp(-1.52554) / (1 + exp(-1.52554))
exp(0.47721) / (1 - control + (control * exp(0.47721)))

[1] 1.452841

First, a note on logistic regression.

The author is fitting a linear model as follows:

$$
\begin{aligned}
 Y = &(-1.52) + 0.22 \cdot Female + (-1.35) \cdot Over45 + (-1.30) \cdot Under25 + \\
     &0.47 \cdot AfAm + (-0.25) \cdot Asian + (-0.42) \cdot Hispanic + \\
     &(1.39) \cdot NativeAmerican + (-0.82) \cdot RaceOther + \\
     &(0.26) \cdot PriorsCount + (-0.31) \cdot CrimeFactorM + 0.68 \cdot TwoYearRecid
\end{aligned}
$$

To calculate the expected score for any *individual*, we plug in the actual values for the individual. For example, let's say that we want to calculate the likelihood of getting a high score for an African-American woman between the ages of 25 and 45 with 3 prior counts. Here's how we would do that

In [4]:
Female <- 1 # 1 because this is among women
Over45 <- 0
Under25 <- 0
AfAm <- 1 # 1 because we're looking at Af-Ams
Asian <- 0
Hispanic <- 0
NativeAmerican <- 0
RaceOther <- 0
PriorsCount <- 3 # 3 prior counts
CrimeFactorM <- 0
TwoYearRecid <- 0

out <- (-1.52554) + 0.22127 * Female + (-1.35563) * Over45 + (1.30839) * Under25 + 
       (0.47721) * AfAm + (-0.25441) * Asian + (-0.42839) * Hispanic + 
       (1.39421) * NativeAmerican + (-0.82635) * RaceOther +
       (0.26895) * PriorsCount + (-0.31124) * CrimeFactorM + (0.68586) * TwoYearRecid

out

[1] -0.02021

But wait, wasn't this supposed to be a *probability* of someone with those characteristics getting a high score? How can the value be negative? Well, this is the joys of using a logistic regression. We have to apply the [link function](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) to transform it into a probability value:

In [5]:
exp(out) / (1 + exp(out)) 

[1] 0.4949477

So African-American women between the ages of 25 and 45 with 3 prior counts have a 49% likelihood of being predicted to have a high score. Now, let's calculate it for a White woman with the same characteristics.

In [6]:
Female <- 1
Over45 <- 0
Under25 <- 0
AfAm <- 0 # Note that this is now set to 0
Asian <- 0
Hispanic <- 0
NativeAmerican <- 0
RaceOther <- 0
PriorsCount <- 3
CrimeFactorM <- 0
TwoYearRecid <- 0

out <- (-1.52554) + 0.22127 * Female + (-1.35563) * Over45 + (1.30839) * Under25 + 
       (0.47721) * AfAm + (-0.25441) * Asian + (-0.42839) * Hispanic + 
       (1.39421) * NativeAmerican + (-0.82635) * RaceOther +
       (0.26895) * PriorsCount + (-0.31124) * CrimeFactorM + (0.68586) * TwoYearRecid

exp(out) / (1 + exp(out)) 

[1] 0.3781472

Now, we can calculate the "relative risk ratio" by comparing those probabilities: 

In [7]:
0.494947671964961 / 0.372852233686804

[1] 1.327463

Here, we show that African-American women between the ages of 25 and 45 with 3 prior counts are 33% more likely to be assigned a high score than similar white women. 

Now, why did we go through this whole exercise? Beause it turns out that the relative risk ratio *changes* based on what group you're comparing. And, in fact, the ProPublica authors write that African-Americans are **45%** more likely to have a higher score. 

So how did they get to 45%?

Well, let's go back to Cell 17. There, the authors ran this calculation:

```
control <- exp(-1.52554) / (1 + exp(-1.52554))
exp(0.47721) / (1 - control + (control * exp(0.47721)))

# 1.45284086581389
```

This is kind of confusing, but if you'll notice, `-1.52` is just the intercept of the model, and `0.47` is just the coefficient on African-American. So if we were to return to our earlier example, it's the equivalent of comparing these two groups:

In [8]:
# In the intercept-only model, all coefficients are set to 0.
Female <- 0
Over45 <- 0
Under25 <- 0
AfAm <- 0
Asian <- 0
Hispanic <- 0
NativeAmerican <- 0
RaceOther <- 0
PriorsCount <- 0
CrimeFactorM <- 0
TwoYearRecid <- 0

out <- (-1.52554) + 0.22127 * Female + (-1.35563) * Over45 + (1.30839) * Under25 + 
       (0.47721) * AfAm + (-0.25441) * Asian + (-0.42839) * Hispanic + 
       (1.39421) * NativeAmerican + (-0.82635) * RaceOther +
       (0.26895) * PriorsCount + (-0.31124) * CrimeFactorM + (0.68586) * TwoYearRecid


white_prob <- exp(out) / (1 + exp(out))
white_prob

[1] 0.1786472

In [9]:
Female <- 0
Over45 <- 0
Under25 <- 0
AfAm <- 1 # Now we're including the African-American coefficient
Asian <- 0
Hispanic <- 0
NativeAmerican <- 0
RaceOther <- 0
PriorsCount <- 0
CrimeFactorM <- 0
TwoYearRecid <- 0

out <- (-1.52554) + 0.22127 * Female + (-1.35563) * Over45 + (1.30839) * Under25 + 
       (0.47721) * AfAm + (-0.25441) * Asian + (-0.42839) * Hispanic + 
       (1.39421) * NativeAmerican + (-0.82635) * RaceOther +
       (0.26895) * PriorsCount + (-0.31124) * CrimeFactorM + (0.68586) * TwoYearRecid


afam_prob <- exp(out) / (1 + exp(out))
afam_prob

[1] 0.2595459

And to make sure we've replicated the risk ratio correctly...

In [10]:
afam_prob / white_prob

[1] 1.452841

Yay, same as what the author calculated! 

But what did we actually do? By comparing the Intercept-Only model versus Intercept+Afam Model, we actually made a **very specific** comparision, namely one where all the over covariates are set to zero. In effect, the author is comparing the likelihood of being classified as high risk for Whites vs. African-American American

* males AND
* between the ages of 25 and 45 AND 
* with no recidivism AND 
* with no priors AND 
* a particular crime severity.

That's actually a really specific group of people! And how many people actually fall into this category?

In [11]:
# Everyone who meets the criteria listed above
meet_criteria <- df$gender_factor != 'Female' & df$age_factor != 'Greater than 45' & 
                    df$age_factor != 'Less than 25' & 
                    df$race_factor %in% c('Caucasian', 'African-American') &
                    df$priors_count == 0 & df$crime_factor != 'M' & df$two_year_recid == 0

# Everyone who is White or Black
all_afam_white <- df$race_factor %in% c('Caucasian', 'African-American')

# Proportion who meet criteria
sum(meet_criteria) / sum(all_afam_white)

[1] 0.04566124

The reported 45% increase in likelihood only applies to **4.6%** of the sample. The fact that the increase is that stark for even a small proportion of the sample is meaningful, but if we actually calculate the value for the entire dataset, the findings are somewhat different.

There are two ways to calculate something closer to the average marginal effect over the entire dataset: 

1. Calculate [marginal effects and levels using the delta method](https://cran.r-project.org/web/packages/modmarg/vignettes/delta-method.html)

2. Calculate the ordinary least squares model. This may lead to model spitting back values greater than 0 or 1 for some combinations of covariates, but the coefficient itself will represent the marginal value. This means that OLS is a bad choice for calculating the risk ratio, but it will be a good sanity check for us.

(And to be clear, these aren't the only ways to calculate effects. You could add interactions to the model, or use non-linear model. But within the logistic regression framework that the authors are trying to use, there are more accurate / representative ways of calculating average marginal effects.)

## Marginal Effects / Levels Using the Delta Method

The basic idea behind a marginal effects model is that it calculates the risk level across *all* covariates based on instead of just using a specific sample. We do this by taking the entire dataset and basically comparing what the model predicts when we set everyone's race to "African-American" versus if we set everyone's race to "White":

In [12]:
# Set Everyone's Race to White
everyone_white <- df %>%
                    mutate(race_factor = 'Caucasian')
# Get Model Predictions
pred_wht <- predict(model, newdata = everyone_white, type = 'response')

# Set Everyone's Race to Af-Am
everyone_afam <- df %>%
                   mutate(race_factor = 'African-American')
# Get Model Predictions
pred_afam <- predict(model, newdata = everyone_afam, type = 'response')

In [13]:
# Average outcomes for white, holding ALL covariates constant
mean(pred_wht) 

[1] 0.4142778

Across all covariates (i.e., all combinations of gender and recidivism and crime type) in this sample, if everyone were white, the average likelihood of getting a "High" score is 41%.

In [14]:
# Average outcomes for Blacks, holding ALL covariates constant
mean(pred_afam)

[1] 0.4974868

Across all covariates (i.e., all combinations of gender and recidivism and crime type) in this sample, if everyone were Black, the average likelihood of getting a "High" score is 49%.

In [15]:
# Risk Ratio
mean(pred_afam) / mean(pred_wht)

[1] 1.200853

The risk ratio is 1.2, i.e., across the set of demographics in the sample, if everyone were African-American, we'd expect the likelihood of getting a "High" score to be 20% higher than if everyone were White. This is a difference of approx

This functionality is also available in the `modmarg` package (of which I am the primary author). The nice part about using this package is that it just makes the syantax easier and also incorporates standard errors on our estimate. Below, we'll calculate these predicted "levels" again. You'll note that the values are identical to above.

In [16]:
modmarg::marg(model, 'race_factor', type = 'levels')

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
race_factor = Caucasian,0.4142778,0.009330919,44.398395,0.000000e+00,0.3959896,0.4325661
race_factor = African-American,0.4974868,0.007593035,65.518830,0.000000e+00,0.4826047,0.5123689
race_factor = Asian,0.3717285,0.077824589,4.776491,1.783804e-06,0.2191951,0.5242619
race_factor = Hispanic,0.3437204,0.018345724,18.735721,2.531908e-78,0.3077634,0.3796773
race_factor = Native American,0.6569112,0.126511111,5.192518,2.074686e-07,0.4089540,0.9048685
race_factor = Other,0.2838012,0.021795958,13.020817,9.316845e-39,0.2410819,0.3265205


Now we'll calculates the *change* in probability for different races, with Caucasian as a baseline, holding other variables constant across all iterations.

In [17]:
modmarg::marg(model, 'race_factor', type = 'effects')

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
race_factor = Caucasian,0.00000000,0.00000000,NaN,NaN,0.00000000,0.00000000
race_factor = African-American,0.08320895,0.01217533,6.834226,8.244875e-12,0.05934574,0.10707215
race_factor = Asian,-0.04254938,0.07835115,-0.543060,5.870885e-01,-0.19611482,0.11101606
race_factor = Hispanic,-0.07055748,0.02052939,-3.436902,5.884092e-04,-0.11079434,-0.03032063
race_factor = Native American,0.24263341,0.12687936,1.912316,5.583570e-02,-0.00604557,0.49131239
race_factor = Other,-0.13047665,0.02363606,-5.520238,3.385413e-08,-0.17680247,-0.08415083


Here the marginal effect of being classified as high risk is 8.3pp higher than for Whites. `0.4974868` - `0.4142778` = `0.08320895`. This difference *is* statistically significant.

The risk ratio, as calculated earlier, is 20% (`0.4974868` / `0.4142778`)

## Ordinary Least Squares

In addition to running marginal effects, we can also do something a bit more low-tech and just run an ordinary least squares (OLS) regression to double-check whether the calculations above make sense. In an OLS regression, the value of the coefficient on African-American will be *constant* across the entire dataset.

In [18]:
# First, change the outcome variable to numeric
df$score_factor_numeric <- as.numeric(df$score_factor == 'HighScore')

model_ols <- glm(score_factor_numeric ~ gender_factor + age_factor + race_factor +
                 priors_count + crime_factor + two_year_recid, family="gaussian", data=df)
summary(model_ols)


Call:
glm(formula = score_factor_numeric ~ gender_factor + age_factor + 
    race_factor + priors_count + crime_factor + two_year_recid, 
    family = "gaussian", data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.13742  -0.32725  -0.06756   0.35213   1.06124  

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.221120   0.013060  16.931  < 2e-16 ***
gender_factorFemale          0.031481   0.013578   2.319   0.0205 *  
age_factorGreater than 45   -0.190708   0.013707 -13.913  < 2e-16 ***
age_factorLess than 25       0.257228   0.013794  18.648  < 2e-16 ***
race_factorAfrican-American  0.096820   0.012125   7.985 1.66e-15 ***
race_factorAsian            -0.043872   0.074869  -0.586   0.5579    
race_factorHispanic         -0.069479   0.020487  -3.391   0.0007 ***
race_factorNative American   0.250573   0.125053   2.004   0.0451 *  
race_factorOther            -0.130449   0.024129  -5.406

In this OLS model, the "effect" of being African-American is 9.7 percentage points, which is comparable to what we got when looking at the marginal effects model. 

# Claim 2: Women are 19.4% more likely than men to get a higher score.

Using the same technique as above, we can replicate the analysis in cell 18. When we do that, women are only 8% more likely.

In [19]:
modmarg::marg(model, 'gender_factor', type = 'levels')

modmarg::marg(model, 'gender_factor', type = 'effects')

0.475 / 0.438 # risk level

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
gender_factor = Male,0.4385519,0.005765158,76.06937,0,0.4272524,0.4498514
gender_factor = Female,0.4753457,0.011845259,40.12961,0,0.4521294,0.4985620


Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
gender_factor = Male,0.00000000,0.00000000,NaN,NaN,0.00000000,0.00000000
gender_factor = Female,0.03679375,0.01321767,2.783679,0.005374628,0.01088759,0.06269991


[1] 1.084475

Women are actually only 8% more likely than men to get a higher score, a difference of under 4 percentage points.

# Claim 3: [P]eople under 25 are 2.5 times as likely to get a higher score as middle aged defendants.

Using the same technique as above, we can replicate the analysis in cell 18. When recalculated, people under 25 are 1.9 times as likely.

In [20]:
modmarg::marg(model, 'age_factor', type = 'levels')

modmarg::marg(model, 'age_factor', type = 'effects')

0.4225 / 0.2207 # risk level

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
age_factor = 25 - 45,0.4225338,0.007226936,58.46652,0.000000e+00,0.4083693,0.4366983
age_factor = Greater than 45,0.2207941,0.010302913,21.43026,6.979636e-102,0.2006007,0.2409874
age_factor = Less than 25,0.6674816,0.011296074,59.08970,0.000000e+00,0.6453417,0.6896215


Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
age_factor = 25 - 45,0.0000000,0.00000000,NaN,NaN,0.0000000,0.0000000
age_factor = Greater than 45,-0.2017397,0.01251396,-16.12117,1.811392e-58,-0.2262667,-0.1772128
age_factor = Less than 25,0.2449479,0.01352305,18.11335,2.500353e-73,0.2184432,0.2714525


[1] 1.914363

People under 25 are 1.9 times as likely, a difference of 20 percentage points.

# Claim 4: The violent score overpredicts recidivism for black defendants by 77.3% compared to white defendants.

When recalculated over the entire sample, this value is only 33%.

In [21]:
# Replicating analysis from cell 20 through 30.

raw_data <- read.csv("./compas-scores-two-years-violent.csv")
nrow(raw_data)

df <- dplyr::select(raw_data, age, c_charge_degree, race, age_cat, v_score_text, sex, priors_count, 
                    days_b_screening_arrest, v_decile_score, is_recid, two_year_recid) %>% 
        filter(days_b_screening_arrest <= 30) %>%
        filter(days_b_screening_arrest >= -30) %>% 
        filter(is_recid != -1) %>%
        filter(c_charge_degree != "O") %>%
        filter(v_score_text != 'N/A')
nrow(df)

df <- mutate(df, crime_factor = factor(c_charge_degree)) %>%
      mutate(age_factor = as.factor(age_cat)) %>%
      within(age_factor <- relevel(age_factor, ref = 1)) %>%
      mutate(race_factor = factor(race,
                                  labels = c("African-American", 
                                             "Asian",
                                             "Caucasian", 
                                             "Hispanic", 
                                             "Native American",
                                             "Other"))) %>%
      within(race_factor <- relevel(race_factor, ref = 3)) %>%
      mutate(gender_factor = factor(sex, labels= c("Female","Male"))) %>%
      within(gender_factor <- relevel(gender_factor, ref = 2)) %>%
      mutate(score_factor = factor(v_score_text != "Low", labels = c("LowScore","HighScore")))
model <- glm(score_factor ~ gender_factor + age_factor + race_factor +
                            priors_count + crime_factor + two_year_recid, family="binomial", data=df)
summary(model)

[1] 4743

[1] 4020


Call:
glm(formula = score_factor ~ gender_factor + age_factor + race_factor + 
    priors_count + crime_factor + two_year_recid, family = "binomial", 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.9304  -0.5667  -0.3161   0.4192   2.8386  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.24274    0.11326 -19.802  < 2e-16 ***
gender_factorFemale         -0.72890    0.12666  -5.755 8.66e-09 ***
age_factorGreater than 45   -1.74208    0.18415  -9.460  < 2e-16 ***
age_factorLess than 25       3.14591    0.11541  27.259  < 2e-16 ***
race_factorAfrican-American  0.65893    0.10815   6.093 1.11e-09 ***
race_factorAsian            -0.98521    0.70537  -1.397   0.1625    
race_factorHispanic         -0.06416    0.19133  -0.335   0.7374    
race_factorNative American   0.44793    1.03546   0.433   0.6653    
race_factorOther            -0.20543    0.22464  -0.914   0.3605    
priors_count  

## Marginal Effects Re-Analysis

In [22]:
modmarg::marg(model, 'race_factor', type = 'levels')
modmarg::marg(model, 'race_factor', type = 'effects')

0.3133580/0.2357338 # risk level

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
race_factor = Caucasian,0.2357338,0.009518571,24.765671,2.101997e-135,0.21707774,0.2543899
race_factor = African-American,0.3133580,0.007895473,39.688312,0.000000e+00,0.29788316,0.3288328
race_factor = Asian,0.1429904,0.056474809,2.531932,1.134359e-02,0.03230181,0.2536790
race_factor = Hispanic,0.2288773,0.018063647,12.670601,8.605207e-37,0.19347318,0.2642814
race_factor = Native American,0.2870232,0.125256298,2.291487,2.193525e-02,0.04152537,0.5325210
race_factor = Other,0.2141940,0.021125177,10.139276,3.698306e-24,0.17278941,0.2555986


Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
race_factor = Caucasian,0.000000000,0.00000000,NaN,NaN,0.00000000,0.00000000
race_factor = African-American,0.077624208,0.01252366,6.1982032,5.711138e-10,0.05307828,0.10217014
race_factor = Asian,-0.092743399,0.05723947,-1.6202700,1.051743e-01,-0.20493070,0.01944390
race_factor = Hispanic,-0.006856523,0.02034080,-0.3370823,7.360548e-01,-0.04672375,0.03301071
race_factor = Native American,0.051289407,0.12560800,0.4083292,6.830320e-01,-0.19489774,0.29747656
race_factor = Other,-0.021539801,0.02308608,-0.9330211,3.508091e-01,-0.06678769,0.02370809


[1] 1.329288

African-Americans are 33% more likely to be classified as high risk, a difference of 7.8 percentage points.

## Ordinary Least Squares Re-Analysis

In [23]:
df$score_factor_numeric <- as.numeric(df$score_factor == 'HighScore')

model_ols <- glm(score_factor_numeric ~ gender_factor + age_factor + race_factor +
                 priors_count + crime_factor + two_year_recid, family="gaussian", data=df)
summary(model_ols)


Call:
glm(formula = score_factor_numeric ~ gender_factor + age_factor + 
    race_factor + priors_count + crime_factor + two_year_recid, 
    family = "gaussian", data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.09342  -0.17992  -0.07787   0.11530   1.02116  

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  0.103822   0.012685   8.184 3.64e-16 ***
gender_factorFemale         -0.073656   0.013556  -5.433 5.86e-08 ***
age_factorGreater than 45   -0.124982   0.013555  -9.220  < 2e-16 ***
age_factorLess than 25       0.596029   0.014732  40.458  < 2e-16 ***
race_factorAfrican-American  0.076099   0.012471   6.102 1.14e-09 ***
race_factorAsian            -0.080923   0.068220  -1.186    0.236    
race_factorHispanic         -0.008205   0.020449  -0.401    0.688    
race_factorNative American   0.037282   0.130611   0.285    0.775    
race_factorOther            -0.020487   0.023430  -0.874

Here, the marginal effect is 7.6 percentage points.

## For what proportion of the sample is the author's comparision valid?

Let's assume that we actually did want to compare an `intercept` vs. `intercept + afam` model. What proportion of African-Americans and Whites actually fall into that category where all other covariates are set to 0? **Only 7.1%.**

In [24]:
meet_criteria <- sum(df$gender_factor != 'Female' & df$age_factor != 'Greater than 45' & 
                     df$age_factor != 'Less than 25' & 
                     df$race_factor %in% c('Caucasian', 'African-American') &
                     df$priors_count == 0 & df$crime_factor != 'M' & df$two_year_recid == 0)

all_afam_white <- sum(df$race_factor %in% c('Caucasian', 'African-American'))

meet_criteria / all_afam_white

[1] 0.07136512

# Claim 5: Defendands under 25 are 7.4 times as likely to get a higher score as middle aged defendants.

When re-calculated, this value is 3.8 times.

In [25]:
modmarg::marg(model, 'age_factor', type = 'levels')

modmarg::marg(model, 'age_factor', type = 'effects')

0.189 / 0.050 # risk level

Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
age_factor = 25 - 45,0.18972413,0.007367728,25.750697,3.165997e-146,0.17528365,0.20416462
age_factor = Greater than 45,0.05037548,0.007230352,6.967224,3.232566e-12,0.03620425,0.06454671
age_factor = Less than 25,0.77612278,0.014546390,53.355010,0.000000e+00,0.74761238,0.80463318


Label,Margin,Standard.Error,Test.Stat,P.Value,Lower CI (95%),Upper CI (95%)
age_factor = 25 - 45,0.0000000,0.00000000,NaN,NaN,0.0000000,0.0000000
age_factor = Greater than 45,-0.1393487,0.01031010,-13.51575,1.262691e-41,-0.1595561,-0.1191412
age_factor = Less than 25,0.5863986,0.01631487,35.94258,6.609330e-283,0.5544221,0.6183752


[1] 3.78

When re-analyzed using the entire sample, Defendants under 25 are only 3.8 times as likely to get a higher score. 